In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import math

%matplotlib inline
data_path = '../data'

In [2]:
train_data_path = os.path.join(data_path,'train.zip')
train_data = pd.read_csv(train_data_path, encoding = "big5")

test_data_path = os.path.join(data_path,'test.zip')
test_data = pd.read_csv(test_data_path, encoding = "big5")

train_data_num = train_data.shape[0]
test_data_txkey = test_data['txkey'].copy()

train_data = train_data.sort_values(by=['locdt','loctm']).reset_index(drop=True)
label_data = train_data['fraud_ind'].copy()

all_data = pd.concat([train_data,test_data],axis=0).reset_index(drop=True)
print(all_data.index)
print(train_data.shape)
print(test_data.shape)
print(all_data.shape)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


RangeIndex(start=0, stop=1943452, step=1)
(1521787, 23)
(421665, 22)
(1943452, 23)


In [3]:
train_data.head()

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6716,12765,101514,966.58,4,62,N,2,N,N,...,2,1,29.0,247,34949,N,5817,102,0,1119159
1,5575,34837,60869,1071.10,5,62,Y,8,N,N,...,0,1,40.0,203,33794,N,2138,44,0,641444
2,6769,22630,110573,438.21,5,62,N,5,N,N,...,0,1,45.0,251,79257,N,3588,102,0,188670
3,6767,91588,208961,1174.17,5,62,N,5,N,N,...,0,1,48.0,293,14765,N,5817,102,0,1475737
4,3198,84148,180103,367.29,5,60,N,5,N,N,...,0,1,51.0,251,68966,N,3982,38,0,641443


In [4]:
print('Missing value training data:\n',train_data.isna().sum()[train_data.isna().sum()>0])
print('Missing value testing data:\n',test_data.isna().sum()[test_data.isna().sum()>0])

## not neccessary to fill null value, since we use lgb model
# all_data.flbmk = all_data.flbmk.fillna(value=all_data.flbmk.mean(skipna=True))
# all_data.flg_3dsmk = all_data.flg_3dsmk.fillna(value=all_data.flg_3dsmk.mean(skipna=True))

Missing value training data:
 flbmk        12581
flg_3dsmk    12581
dtype: int64
Missing value testing data:
 flbmk        3715
flg_3dsmk    3715
dtype: int64


In [5]:
all_data.ecfg = all_data.ecfg.map({'N':0,'Y':1})
all_data.ovrlt = all_data.ovrlt.map({'N':0,'Y':1})
all_data.insfg = all_data.insfg.map({'N':0,'Y':1})
all_data.flbmk = all_data.flbmk.map({'N':0,'Y':1})
all_data.flg_3dsmk = all_data.flg_3dsmk.map({'N':0,'Y':1})
all_data.loctm = all_data.loctm.astype(int)
all_data = all_data.infer_objects()

# print(all_data.dtypes)

## Anomaly detection
* one class svm
* isolation tree
* replicator NN
* KNN(take too much time)

In [6]:
weird1 = (all_data['insfg']==1)&(all_data['iterm']==0)
print(weird1.value_counts())

False    1943422
True          30
dtype: int64


## Feature engineering
* train & valid only

In [7]:
positive_data = all_data[all_data['fraud_ind']==1].copy()
# print(positive_data)

In [9]:
all_data['conam'] = np.log(all_data['conam']+1)
all_data['locdt_week'] = all_data['locdt']%7+1
all_data['locdt_month'] = all_data['locdt']%30+1

all_data['loctm_hr'] = all_data['loctm'].apply(lambda s:s//10000).astype(int)
all_data['loctm_hr2'] = all_data['loctm'].apply(lambda s:s//1000).astype(int)
# all_data['loctm_hr_sin'] = all_data['loctm_hr'].apply(lambda s:math.sin(s/24*math.pi)).astype(int)
# all_data['loctm_hr2_sin'] = all_data['loctm_hr2'].apply(lambda s:math.sin(s/240*math.pi)).astype(int)

mean_df = all_data.groupby(['bacno'])['cano'].nunique().reset_index()
mean_df.columns = ['bacno', 'cano'+'_count']
all_data = pd.merge(all_data, mean_df, on='bacno', how='left')

mean_df = all_data.groupby(['bacno'])['txkey'].nunique().reset_index()
mean_df.columns = ['bacno', 'txkey'+'_count']
all_data = pd.merge(all_data, mean_df, on='bacno', how='left')

mean_df = all_data.groupby(['bacno'])['loctm_hr'].mean().reset_index()
mean_df.columns = ['bacno', 'loctm_hr'+'_mean']
all_data = pd.merge(all_data, mean_df, on='bacno', how='left')

mean_df = all_data.groupby(['bacno'])['loctm_hr'].var().reset_index()
mean_df.columns = ['bacno', 'loctm_hr'+'_var']
all_data = pd.merge(all_data, mean_df, on='bacno', how='left')


In [ ]:
# data = pd.concat([df[:train_num], train_Y], axis=1)
# for c in df.columns:
#     mean_df = data.groupby([c])['SalePrice'].mean().reset_index()
#     mean_df.columns = [c, f'{c}_mean']
#     data = pd.merge(data, mean_df, on=c, how='left')
#     data = data.drop([c] , axis=1)


# all_data['howmany_cano'] = 
# all_data['howmany_txkey'] = 

## bacno刷卡頻率分佈

# all_data['fraud_before'] =
# all_data['fraud_last_time'] =

# 印出某個被盜刷的人的刷卡使用時間分佈

#
該交易的歸戶帳號是否曾經被盜刷
該交易的歸戶帳號是否曾經被盜刷卻又復原
該交易的歸戶帳號是否第一次刷卡
該交易的歸戶帳號第幾次刷卡


該交易的卡號是否曾經被盜刷
該交易的卡號是否曾經被盜刷卻又復原
該交易的卡號是否第一次刷卡
該交易的卡號第幾次刷卡

消費國別是否跟自己所有消費的眾數不一樣
消費城市是否跟自己所有消費的眾數不一樣
消費地幣別是否跟自己所有消費的眾數不一樣
支付型態是否跟自己所有消費的眾數不一樣
分期期數是否跟自己所有消費的眾數不一樣

是否第一次網路消費且過去有非網路消費的經驗



In [ ]:
delete_list = ['bacno','txkey','locdt','loctm','cano','fraud_ind']
binary_list=['ecfg','insfg','ovrlt','flbmk','flg_3dsmk']
category_list=['contp','etymd','hcefg','stocn','scity','stscd','csmcu']


## Parse train,valid

In [ ]:
X_train = all_data[all_data['locdt']<=60].drop(columns=delete_list)
y_train = all_data[all_data['locdt']<=60]['fraud_ind']
X_test = all_data[(all_data['locdt']>60) & (all_data['locdt']<=90)].drop(columns=delete_list)
y_test = all_data[(all_data['locdt']>60) & (all_data['locdt']<=90)]['fraud_ind']

# all_train = all_data[all_data['locdt']<=90]
# all_test = all_data[all_data['locdt']<=90]['fraud_ind']
# test_data = all_data[all_data['locdt']>90]

In [ ]:
random_seed = 33

print(X_train.shape)
print(y_train.sum()/y_train.shape[0])
print(y_test.sum()/y_test.shape[0])


import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score

def lgb_f1_score(y_true, y_pred):
    y_pred = np.round(y_pred) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_pred), True

param_dist_lgb = {
#                   'num_leaves':25, 
#                   'max_depth':5, 
                  'learning_rate':0.1, 
                  'n_estimators':3500,
                  'objective': 'binary',
#                   'subsample': 1, 
#                   'colsample_bytree': 0.5, 
#                   'lambda_l1': 0.1,
#                   'lambda_l2': 0,
#                   'min_child_weight': 1,
                  'random_state': random_seed,
                 }
evals_result = {}

lgb_clf = LGBMClassifier(**param_dist_lgb)
lgb_clf.fit(X_train, y_train,
        eval_set=[(X_train, y_train),(X_test, y_test)],
        eval_metric=lgb_f1_score,
        early_stopping_rounds=600,
        verbose=True,
        callbacks=[lgb.record_evaluation(evals_result)]
        )

print('F1',f1_score(y_test, lgb_clf.predict(X_test)))

In [ ]:
print('Plotting metrics recorded during training...')
ax = lgb.plot_metric(evals_result, metric='f1')
plt.show()

print('Plotting feature importances...')
ax = lgb.plot_importance(lgb_clf, max_num_features=10)
plt.show()

print('Plotting 4th tree...')  # one tree use categorical feature to split
ax = lgb.plot_tree(lgb_clf, tree_index=3, figsize=(15, 15), show_info=['split_gain'])
plt.show()

print('Plotting 4th tree with graphviz...')
graph = lgb.create_tree_digraph(lgb_clf, tree_index=3, name='Tree4')
graph.render(view=True)

## write csv

In [ ]:
# lgb_clf = LGBMClassifier(**param_dist_lgb)
# lgb_clf.fit(train_data,label_data)

# result = lgb_clf.predict(test_data)
# print(result.sum())
# print(result.sum()/result.shape[0])
# print(label_data.sum()/label_data.shape[0])

# test_data_txkey = test_data['txkey'].copy()

# import csv
# with open('../prediction/submit_lgb.csv','w') as f:
#     writer = csv.writer(f)
#     writer.writerow(['txkey','fraud_ind'])
#     for i in range(result.shape[0]):
#         writer.writerow([test_data_txkey[i], result[i]])

In [ ]:
def analze_distribution(data, target_col, feature, data_test):
#     if data[feature].nunique()!=data_test[feature].nunique():
#         print('data nunique not the same')
    
    mean_data = data.groupby(feature)[target_col].mean()
    mean_data_test = data_test.groupby(feature)[target_col].mean()
    distribution_data = data[feature].value_counts(dropna=False)
    distribution_data_test = data_test[feature].value_counts(dropna=False)
    
    fig, axs = plt.subplots(2,2,figsize=(10,10))
    axs[0,0].plot(mean_data.index, mean_data.values, marker='o')
    axs[0,0].set_title('Average {} wrt {}'.format(target_col,feature))
    axs[0,0].set_ylabel('mean of {}'.format(target_col))
    axs[0,0].set_xlabel(feature)
    axs[0,1].bar(distribution_data.index, distribution_data.values, alpha=0.5)
    axs[0,1].set_title('distribution of {}'.format(feature))
    axs[0,1].set_ylabel('count of {}'.format(feature))
    axs[0,1].set_xlabel(feature)
    
    # Add text in figure coordinates
    plt.figtext(0.5, 1,   'Train data plot', ha='center', va='center', fontsize=15)
    plt.figtext(0.5, 0.5, 'Test data plot', ha='center', va='center', fontsize=15)
    axs[1,0].plot(mean_data_test.index, mean_data_test.values, marker='o')
    axs[1,0].set_title('Average {} wrt {}'.format(target_col,feature))
    axs[1,0].set_ylabel('mean of {}'.format(target_col))
    axs[1,0].set_xlabel(feature)
    axs[1,1].bar(distribution_data_test.index, distribution_data_test.values, alpha=0.5)
    axs[1,1].set_title('distribution of {}'.format(feature))
    axs[1,1].set_ylabel('count of {}'.format(feature))
    axs[1,1].set_xlabel(feature)
    plt.tight_layout(pad=3)


In [ ]:
train_d =pd.concat([X_train,y_train],axis=1)
valid_d =pd.concat([X_test,y_test],axis=1)
print(train_d.csmcu.value_counts())
print(valid_d.csmcu.value_counts())

# analze_distribution(train_d,'fraud_ind','hcefg',valid_d)
# analze_distribution(train_d,'fraud_ind','csmcu',valid_d)
analze_distribution(train_d,'fraud_ind','loctm_hr',valid_d)

# analze_distribution(train_d,'fraud_ind','flbmk',valid_d)
# analze_distribution(train_d,'fraud_ind','hcefg',valid_d)
# analze_distribution(train_d,'fraud_ind','csmcu',valid_d)
# analze_distribution(train_d,'fraud_ind','flg_3dsmk',valid_d)

from featexp import get_univariate_plots
# get_univariate_plots(data=train_d, target_col='fraud_ind', features_list=['hcefg'], bins=100, data_test=valid_d)
# get_univariate_plots(data=train_d, target_col='fraud_ind', features_list=['loctm_hr'], bins=10, data_test=valid_d)
# get_univariate_plots(data=train_d, target_col='fraud_ind', features_list=['flg_3dsmk'], bins=10, data_test=valid_d)
# get_univariate_plots(data=train_d, target_col='fraud_ind', features_list=['csmcu'], bins=100, data_test=valid_d)



In [ ]:
def describe(train_data,target_name):
    target = train_data[target_name]
    print(target_name)
    print('nunique',target.nunique())
#     print('max',target.max())
#     print(target.value_counts())
    bins = target.nunique() if target.nunique()<100 else 100
    target.hist(bins=bins)
    plt.show()


for c in train_data.columns:
    describe(train_data,c)